### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

### Чтение файла

In [2]:
df = pd.read_csv('corp_lemm.csv')

In [3]:
df = df[df['text'].notnull()]
df = df.drop(['Unnamed: 0'], axis = 1)

In [4]:
df.head(6)

,text,rated,coloring,date,name,bank,sex,text_lemmatized
0,Возмутительный способ обмана придумали крючкот...,1,bad,"04 апр, 2022",Сергей,vtb,male,возмутительный способ обман придумать крючкотв...
1,"Не храните в нем деньги, ни коем случае! У мен...",1,bad,"30 мар, 2022",Альбина,uralsib,female,не хранить немой деньга кой случай у карта сч...
2,С 09.03.22 не могу получить средства с евро сч...,1,bad,"24 мар, 2022",Ирина,uralsib,female,с мочь получить средство евро счёт должный выд...
3,"Очень долгое обслуживание, сотрудники совершен...",1,bad,"14 мар, 2022",Александр,gazprombank,male,очень долгий обслуживание сотрудник совершенно...
4,Звоню третий день по вопросу валюты. Оператор ...,1,bad,"14 мар, 2022",Сергей Владимирович,gazprombank,unkn,звонить третий день вопрос валюта оператор го...
5,"Опыт сложился негативный, Сегодня 3марта 2022 ...",1,bad,"03 мар, 2022",Илья,gazprombank,male,опыт сложиться негативный сегодня март год сра...


### Векторизация

In [5]:
nlp = [nltk.word_tokenize(i) for i in df['text_lemmatized']]

In [6]:
model = gensim.models.Word2Vec(nlp, vector_size=500, window=5, min_count=2, sg=0)

In [7]:
model.init_sims(replace=True)
model.save('my.model')

<ipython-input-7-f65a0fe831e7>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [8]:
from gensim.models import Word2Vec
m=Word2Vec.load('my.model')

In [9]:
print('cbow','\n')
for t in m.wv.most_similar(positive=['карта'], topn=1000):
    print (t[0], t[1])

cbow 

дебетовый 0.9488017559051514
халва 0.8883744478225708
обзавестись 0.8656035661697388
перевести 0.8652600049972534
совмещать 0.8652015328407288
мультивалютный 0.8632315397262573
открыть 0.8626410961151123
совершенный 0.8614335060119629
замешкаться 0.8605849742889404
неименной 0.8598144054412842
эскроу 0.8584582209587097
история 0.8559949994087219
пополнять 0.8510608673095703
дебитовый 0.8510423302650452
вкратце 0.8484678268432617
зачисление 0.8468352556228638
несложно 0.8463945388793945
закрыть 0.8432660102844238
расчётный 0.843136727809906
накопительный 0.8427742719650269
выписать 0.8411373496055603
полноценно 0.8403483629226685
перевод 0.8402972221374512
новикомбанк 0.839368462562561
пополнение 0.8390868902206421
мастер 0.8367301225662231
брокерский 0.834944486618042
арестовать 0.8348407745361328
пользоваться 0.8343115448951721
сбербанк 0.830643892288208
эквивалент 0.8302713632583618
банковский 0.8298023343086243
комисия 0.8295625448226929
лимит 0.8294107913970947
пополнить 0.8

In [10]:
w2v = dict(zip(m.wv.index_to_key, model.wv.vectors))

In [11]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [12]:
data_mean=mean_vectorizer(w2v).fit(df['text_lemmatized']).transform(df['text_lemmatized'])
data_mean.shape

(4116, 500)

In [13]:
df['vector'] = data_mean.tolist()

### Анализ тональности

In [14]:
sentiment_mapping = {'bad': 0, 'neutral': 0, 'good': 1}
df['target'] = df['coloring'].map(sentiment_mapping)
df['target'].value_counts()

1    2120
0    1996
Name: target, dtype: int64

In [15]:
df_train, df_test = train_test_split(df[['text', 'target', 'coloring', 'vector','rated']].copy(), test_size=0.3)

In [16]:
x_train, x_test = np.array(df_train['vector'].to_list()), np.array(df_test['vector'].to_list())
y_train, y_test = df_train['target'], df_test['target']
clf = LogisticRegression(multi_class='ovr', class_weight='balanced', max_iter=10000)
clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=10000, multi_class='ovr')

In [17]:
y_pred_train, y_pred_test = clf.predict_proba(x_train), clf.predict_proba(x_test)

In [18]:
df_train.insert(0, 'predictions', [x.round(3).tolist() for x in y_pred_train])
df_test.insert(0, 'predictions', [x.round(3).tolist() for x in y_pred_test])
df_test['predictions'] = [x.round(3).tolist() for x in y_pred_test]
y_pred_train_label, y_pred_test_label = y_pred_train.argmax(axis=1), y_pred_test.argmax(axis=1)


### Оценка качества работы алгоритма

In [19]:
def multiclass_roc_auc(y_true, y_pred):
    if len(set(y_true)) == 2:
        r = roc_auc_score(y_true, y_pred[:, 1])
    else:
        r = roc_auc_score(y_true, y_pred, multi_class='ovr')
    return r
print('ROC-AUC:\n')
print(multiclass_roc_auc(y_train, y_pred_train))
print(multiclass_roc_auc(y_test, y_pred_test))

ROC-AUC:

0.6943801187195578
0.6634023972962193


In [20]:
pred = []
for i in df_test['predictions']:
    if i[0]>i[1]:
        pred.append('neg')
    else:
        pred.append('pos')
tr = []
for i in df_test['coloring']:
    if i == 'good':
        tr.append('pos')
    else:
        tr.append('neg')
df_test = df_test.drop(['coloring', 'target','vector'], axis=1)
df_test['true'] = tr
df_test['predicted'] = pred

In [21]:
a = []
b = []
for i in df_test['predicted']:
    a.append(i)
for i in df_test['true']:
    b.append(i)

In [22]:
tp = 0 
tn = 0 
fp = 0 
fn = 0
t_f = []
for i in range(0, len(a)):
    if a[i] == b[i]:
        if a[i] == 'pos':
            tp+=1
            t_f.append('tp')
        else:
            tn+=1
            t_f.append('tn')
    elif a[i] != b[i]:
        if a[i] == 'pos':
            fp+=1
            t_f.append('fp')
        else:
            fn+=1
            t_f.append('fn')
        

In [23]:
df_test['result'] = t_f

In [24]:
accuracy = (tp+tn)/len(df_test)
precision = tp/ (tp+fp)
recall = tp/(tp+fn)
f1 = 2/(1/precision+1/recall)
f_measure = 2*((precision*recall)/(precision+recall))

In [25]:
print('accuracy', accuracy)
print('precision', precision)
print('recall', recall)
print('f1-score', f1)
print('f-measure', f_measure)

accuracy 0.6299595141700405
precision 0.6576576576576577
recall 0.5775316455696202
f1-score 0.6149957877000842
f-measure 0.6149957877000842


In [26]:
t_res = df_test.query('result == "tn" or result == "tp"')
f_res = df_test.query('result == "fn" or result == "fp"')
pd.options.display.max_colwidth = 10000

In [27]:
f_res.query('result == "fp"')  #ложноположительные отзывы

,predictions,text,rated,true,predicted,result
335,"[0.481, 0.519]","Доброго времени суток. В феврале 2019 года открыла ИП, решила открыть расчетный счёт в Модульбанка, понравились условия по открытию, обслуживанию расчетного счета. В декабре 2019 года оплатила тариф Оптимальный Навсегда. Весь год прекрасное обслуживание, создалось впечатление, что надёжный банк, всё здраво с пониманием к моим проблемам, в 2020 году подключила онлайн бухгалтерию, полное понимание безопасности, доверяю банку. В декабре 2020 года пришлось закрыть ИП, а с 2021 года начались чудеса, БАНК БЕЗ УВЕДОМЛЕНИЯ МЕНЯ О СНЯТИИ МОИХ ДЕНЕЖНЫХ СРЕДСТВ, пока я ждала от налоговой камеральной проверки по закрытию ИП, ЗАБРАЛ ВСЕ МОИ ДЕНЬГИ, которые я оставляла на закрытие налогов. Я РАЗОЧАРОВАНА В Модульбанке! Чувствую, что меня ПРЕДАЛИ и ОБМАНУЛИ!",1,neg,pos,fp
823,"[0.489, 0.511]","В декабре 2020 г. произвели частично досрочное погашение ипотечного кредита на очень хорошую сумму при условии уменьшения ежемесячного платежа чему есть подтверждение в виде заявления на бумажном носителе. Вдруг в январе выясняется, что менеджер банка самостоятельно поменяла условие уменьшения ежемесячного платежа на погашение с уменьшением срока. Чем вызвано такое решение, кроме как желанием менеджера получить премию в какун НГ я обьяснить не могу,как не могу и обьяснить как такое вообще воозможно. Но все обстоит именно так :((.Написала претензию с просьбой устранить это небольшое недоразумение....Было это 21 января.В ответ тишина...обращаюсь в банк практически еженедельно, девочки менеджеры пожимают плечами-ждите! Может когда и решится. Теперь собираем бумаги чтобы подать в суд. Менеджеры которы принимали заявление и сумели его так извратить работают в отделение банка на Крупской. Фамилии пока не называю, жду пару дней прежде чем начать везде публиковать свою историю общения с этим банком. Очень не рекомендую данный банк, за ошибки банка именно Вы заплатите своими деньгами, нервами и временем.",2,neg,pos,fp
623,"[0.462, 0.538]","В 2017 году оформил кредитку «Все сразу». Первые 2 месяца пользования приходили выписки на почту с расшифровкой операций по карте, но затем банк заменил это общей информацией по задолженности и чтобы посмотреть мои расходы за месяц нужно было заходить в личный кабинет. Это я благополучно и делал до сегодняшнего дня, проверяя мои расходы и транзакции. Но сегодня обнаружил, что за предыдущий месяц с меня удержали около 1000 руб за операцию Raiffeisen LIFE (PPI) max. Ранее ничего подобного не было в личном кабинете. Звоню в банк, выясняю в чем дело. Оказывается при оформлении я сам того не зная (не читая договор) подписал страхование жизни, причём списание ежемесячное. На мой вопрос почему мне ранее не приходили суммы списания в личном кабинете, девушка мне ответила, что наблюдались технические сбои в работе. 3 года технических сбоев!!! и что я могу оставить обращение чтобы мне все устранили и я получал эту информацию своевременно. Через несколько минут я заново вошёл в свой личный кабинет и увидел все списания за предыдущие годы, каждый месяц, исправно)) Очень «хороший и честный» банк, буду закрывать все карты и счета, так себя не ведут. Причём ещё раз обращаю внимание, что когда выписки приходили на почту в бумажном виде, данные списания не производились, а когда их убрали и обобщили, списания начались, а в личном кабинете как раз в этот момент случился сбой. Если кто-то вдруг не знал об этом, проверьте, возможно и вы будете удивлены.<br/> Выводы делайте сами, о лояльности такому банку тоже ещё раз подумайте.",1,neg,pos,fp
1984,"[0.5, 0.5]","Вчера прочитала в кредитном договоре с банком о пени в размере 0,6% в день от суммы неоплаченных обязательств. Касается ли это КАСКО, должна ли я ее все таки оформить и оплатить просрочку? Первоначальный полис закончился 20.12.2014г.",3,neg,pos,fp
918,"[0.495, 0.505]","При пополнении карты через банкомат (в отделении Райффайзенбанка) зачислилась сумма на 10 000 рублей меньше, чем вносил. Заметил сразу, не отходя от банкома

In [28]:
f_res.query('result == "fn"') #ложноотрицательные отзывы

,predictions,text,rated,true,predicted,result
3780,"[0.551, 0.449]","Неоднократно была в банке, все быстро, операторы вежливые, отличный банк по обслуживанию клиентов. Так же открыли счёт, как Юр лица",5,pos,neg,fn
2534,"[0.549, 0.451]","Здравствуйте. Я клиент вашего гаспромбанка зарплатная карта в нем. По обслуживанию вопросов нет все нормально, друзьям советую. 09.09.2019г. пригласили в банк и вручили дебетовую карту, сказали в течении недели пришлем СМС оповещение. Тишина. Смысл выпускать карту вручать ее а после проверять стоит довать человеку деньги или нет? (ОБЕЩАТЬ НЕ ЗНАЧИТ ЖЕНИТСЯ). Потратил время.",4,pos,neg,fn
2508,"[0.543, 0.457]","В целом банком доволен! Хочу вырозить благодарность, это единственный банк который сделал мне рефинансироваение кредитов. Не однократно обращался в разные банки и все ответили отказом. Данный банк тоже не с первого раза одобрил кредит но все же с 4 попытки одобрели, условия не совсем конечно такие как обещали, а именно процентрая ставка по рефинансированию у меня составила 18.9, но это без страховок, но все равно это очень много для рефинансирования. но так как это было на много выгоднее чем мои прошлые кредиты я естественно думать не стал. В целом большое спасибо банку.",4,pos,neg,fn
2527,"[0.537, 0.463]","взят кредит в Газпромбанке,в 1,10,2019 обьявили о комиссии 300р.за оплату кредита ,сотрудница сделала карту -для оплатыкредита без комиссии,но при визите в банк было написано заявление на ДПК .деньги в этот день я перевела на счет с карты и спросила у нее правильно ли я сделала,потому что первый раз,на что она сказала что да?!На следующий день я в личном кабинете увидела у себя ДОЛГ,т.е.эти деньги ,которые лежали на счете не списались,позвонив в банк,узнала ,что деньги списываются с карты,и за это день просрочки ПЕННИ в районе 5000,хотя писала заявление на досрочное и ездила в банк,получается что сотрудник сознательно обманул и по телефону сотрудник банка рассказавм ей об этом сказала что бывает!!!",4,pos,neg,fn
3053,"[0.508, 0.492]","Подробно рассказали об условиях, подобрали более выгодный продукт. Деньги перечислили моментально. Если возникают вопросы отвечают быстро в чате.",5,pos,neg,fn
...,...,...,...,...,...,...
3434,"[0.507, 0.493]","Достаточно долго пользуюсь продуктом Халва, пока что очень доволен, сервис работает стабильно, пока нареканий нет. Надеюсь и далее будет также.",5,pos,neg,fn
2553,"[0.501, 0.499]","Я давно являюсь клиентом Росбанка, и кредит и рефинансирования у меня в этом банке. Но столкнулась с такой ситуацией, так как мой любимый банк я обратилась за рефинансированием туда. Девушки специалист надавила на то, что банк одобрил и надо быстрее оформляться. У меня не было времени и выбора, т.к надо расплатиться быстрее с другими банками, чтобы процентов лишних не начислялись( После расчета суммы ежемесячной платы, мне показалось большая сумма, на что мне ответили что % повысились и везде так! Я все взвесила и решила оформляться выбора все равно не было. При подписания договора, оказывается что мне включили страховку и не сказали. После моего возмущения, девушка сказала что это обязательно а то откажут и больше в этом банке вам кредит не дадут и вернуть вы тоже не имеете право, а то заставят вернуть весь кредит в полном размере, якобы такие условия банка. Меня загнали в угол, боюсь предпринимать меры по возврату страховки( хотела вылезти из долгов а мне еще повесили. Единственный вариант который я вижу обращаться в суд.",4,pos,neg,fn
3390,"[0.541, 0.459]","Нормальная кредитка, пользуюсь лет пять, уже второй раз взял кредит с возвратом процентов. После того как выплатил первый всё вернулось на второй день. Переплачиваешь только за финзащиту",5,pos,neg,fn
2112,"[0.542, 0.458]","Регулярно совершаю покупки и получаю кэшбек от 200 до 3000 в мес. в зависимости от покупок, а также приятный бонус - проценты на остаток по счету.",4,pos,neg,fn


### Общие выводы

Видим значительное снижение точности анализа. F-мера приняла значение 0.65, что на 0.21 ниже, чем при векторизации по метрике TF-IDF, при этом снизились оба показателя – полноты и точности. Метрика ROC-AUC упала на 0.3.

Векторизация текстов методом Skip-gram происходит аналогично, в методе gensim.models.Word2Vec() достаточно изменить параметр ‘sg’ на единицу, вместо нуля. Значения метрик качества анализа при векторизации методом Skip-gram:

- Roc-auc(train)0.7486313472177969 
- Roc-auc(test) 0.7503461804754211
- accuracy 0.697165991902834
- precision 0.7264325323475046
- recall 0.6348949919224556
- f1-score 0.6775862068965517
- f-measure 0.6775862068965518

Видим небольшие улучшения по всем метрикам по сравнению с моделью CBOW, однако алгоритм все еще сильно уступает методу с векторизацией по значению TF-IDF.
